## Using Markov Chain Monte Carlo for Code-Breaking
_Ec 143, Spring 2026_
_Bryan S. Graham, UC - Berkeley_

This notebook illustrates how an analyst can use Markov Chain Monte Carlo (MCMC) methods to "crack" a simple substitution cipher. The example is based upon work reaccounted in the paper:

Diaconis, Persi. (2009). "The Markov chain Monte Carlo revolution," _Bulletin of the American Mathematical Society_ 46 (2): 179 - 205.

Some of the code found below was adapted from material found on [this](https://github.com/mattreyes/mcmc-substitution-cipher/tree/master) GitHub repository.

The example is of interest to us for several reasons. First, it provides a novel example of how "thinking like a Bayesian" can help one find solutions to seemingly intractable problems. Second, simulation methods feature widely in modern econometric analysis. Markov Chain Monte Carlo (MCMC) is a leading example. Here we use the foundational Metropolis-Hastings Algorithm. Third, the example highlights connections between stochastic search/optimization and posterior sampling.

While you unlikely to engage in code-breaking as part of your day job as an economists, you may very well utilize the tools employed below in your work. 

#### Code citation:
<br>
Graham, Bryan S. (2025). "Using Markov Chain Monte Carlo for Code-Breakings Python Jupyter Notebook," (Version 1.0) (Computer program). Available at http://bryangraham.github.io/econometrics/ (Accessed 27 October 2025)    

In [1]:
import re 
from collections import Counter
import random
import numpy as np
import pandas as pd

In [2]:
def encrypt(plaintext, plaintext_alphabet, cipher_alphabet, blacklist={}):
    
    """
    This function takes a plaintext and returns an encrypted ciphertext.
    
    INPUTS
    ------
    plaintext          : plaintext to be encrypted (string)
    plaintext_alphabet : alphabet of language plain text is written in (list)
    cipher_alphabet    : corresponding alphabet of ciphertext; the mapping from plaintext_alphabet to 
                         cipher_alphabet is one-to-one with the first element of plaintext_alphabet 
                         mapping into the first element of cipher_alphabet etc. (list)
    blacklist          : list of characters that are not encrypted                   
    
    OUTPUTS
    -------
    ciphertext         : encrypted version of plaintext (string)
    
    SOURCES 
    -------
    Some of the code is this function was adapted from material in this GitHub repository:
    
    https://github.com/mattreyes/mcmc-substitution-cipher/tree/master
    
    """
    
    # Check that the plaintext only contains characters in plaintext alphabet. First remove blacklist elements
    # from plaintext, then check that balance of characters can all be found in plaintext_alphabet.
    assert set(list(plaintext)).difference(blacklist).issubset(set(list(plaintext_alphabet))) 
    
    # Check that the cardinality of cipher_alphabet and plaintext_alphabet coincide
    assert len(plaintext_alphabet) == len(cipher_alphabet) 

    # Create a dictionary with the plaintext_alphabet elements as keys and cipher_alphabet elements as values
    encyption_key = dict(zip(list(plaintext_alphabet),list(cipher_alphabet)))
    
    # Encrypt the plaintext
    scc = []                            # initialize list for storing ciphertext
    for l in plaintext:                 # iterate through characters in plaintext
        if l in blacklist:              # leave characters in blacklist "as is" 
            scc += l
        else:                           # swap all other characters using the enc_key dictionary
            scc += [encyption_key[l]]
    ciphertext = "".join(scc)           # convert encrypted ciphertext into a string

    return ciphertext
           

def decrypt(ciphertext, plaintext_alphabet, cipher_alphabet, blacklist={}):
    
    """
    
    This function takes a ciphertext and returns an decrypted plaintext.
    
    INPUTS
    ------
    ciphertext         : ciphertext to be decrypted (string)
    plaintext_alphabet : alphabet of language plain text is written in (list)
    cipher_alphabet    : corresponding alphabet of ciphertext; the mapping from plaintext_alphabet to 
                         cipher_alphabet is one-to-one with the first element of plaintext_alphabet 
                         mapping into the first element of cipher_alphabet etc. (list)
    blacklist          : list of characters that are not encrypted                   
    
    OUTPUTS
    -------
    ciphertext         : encrypted version of plaintext (string)
    
    SOURCES 
    -------
    Some of the code is this function was adapted from material in this GitHub repository:
    
    https://github.com/mattreyes/mcmc-substitution-cipher/tree/master
    
    """
    
    # Create a dictionary with the cipher_alphabet elements as keys and plaintext_alphabet elements as values
    decryption_key = dict(zip(list(cipher_alphabet),list(plaintext_alphabet)))
    
    # decrypt the ciphertext
    scc = []                            # initialize list for storing plaintext
    for l in ciphertext:                # iterate through characters in ciphertext
        if l in blacklist:              # leave characters in blacklist "as is" 
            scc += l
        else:                           # swap all other characters using the dec_key dictionary
            scc += [decryption_key[l]]
    plaintext = "".join(scc)            # convert decrypted ciphertext into a string
    
    return plaintext

In [3]:
def process_text(filename, regex_ignore='[^A-Z .]'):
    
    """
    This function takes a reference text and returns trigram, bigram and unigram frequences. 
    These can be used to construct a decryption score function.
    
    INPUTS
    ------
    filename       : reference text filename
    regex_ignore   : the default value of '[^A-Z .]' indicates matching with _any_ character that is _not_            
                     uppercase, a space or a period. 
        
    OUTPUTS
    -------

    reference_freq : dictionary of dictionaries with trigram, bigram and unigram frequences found in reference text
    
    SOURCES 
    -------
    Some of the code is this function was adapted from material in this GitHub repository:
    
    https://github.com/mattreyes/mcmc-substitution-cipher/tree/master
    
    """
    
    trigram_counts = Counter()                        # containers for counts of various trigrams,
    bigram_counts  = Counter()                        # bigrams and unigrams
    unigram_counts = Counter()

    # read file which contains reference text
    with open(filename, encoding='unicode_escape') as f:
        
        cnt = 0
        
        # iterate through each line in the file
        for line in f:
            
            # keep track of number of lines read and periodically report status
            cnt += 1         
            if cnt % 1000 == 0: print("{0} lines read.".format(cnt))
            
            # clean up line of text
            if regex_ignore != None:
                pattern = re.compile(regex_ignore)
                s = pattern.sub('',line.upper())      # remove any characters that are not letters, spaces
            else:                                     # or periods and capitalize balance of text.
                s = line.upper()                      

            line_length = len(s)                      # calculate length of line and if it is greater than
                                                      # zero count trigrams, bigrams and unigrams
            if line_length > 0:
                for c in range(line_length-2):
                    trigram_counts[(s[c],s[c+1],s[c+2])] +=1
                    bigram_counts[(s[c],s[c+1])] +=1
                    unigram_counts[s[c]] += 1

                # Add last bigram in the line
                unigram_counts[(s[line_length-2],s[line_length-1])] += 1     
                
                # Add last two unigrams in the line
                unigram_counts[s[line_length-2]] += 1 
                unigram_counts[s[line_length-1]] += 1 
                
    # Convert trigram, bigram and unigram counts into frequencies/probabilities
    total_trigrams = sum(trigram_counts.values())
    total_bigrams  = sum(bigram_counts.values())
    total_unigrams = sum(unigram_counts.values())
            
    pop_trigram_freqs = {}
    for trigram, count in trigram_counts.items():
        pop_trigram_freqs[trigram] = count / total_trigrams
        
    pop_bigram_freqs = {}
    for bigram, count in bigram_counts.items():
        pop_bigram_freqs[bigram] = count / total_bigrams
                
    pop_unigram_freqs = {}
    for unigram, count in unigram_counts.items():
        pop_unigram_freqs[unigram] = count / total_unigrams        

    reference_freqs = {'pop_trigram_freqs': pop_trigram_freqs, 'pop_bigram_freqs': pop_bigram_freqs,\
                      'pop_unigram_freqs': pop_unigram_freqs}
  
    return reference_freqs

In [4]:
def composite_ciper_logl(ciphertext, plaintext_alphabet, cipher_alphabet, reference_freq, blacklist={}):

    """
    This function constructs the composite log-likelihood for the observed ciphertext based on given
    cipher_alphabet-to-plaintext_alphabet mapping. The composite log-likelihood is constructed based upon
    unigrams, bigrams and trigrams.
    """
    
    trigram_counts = Counter()                        # containers for counts of various trigrams,
    bigram_counts  = Counter()                        # bigrams and unigrams
    unigram_counts = Counter()
    
    epsilon = np.finfo(float).eps
    
    # Decrypt cipertext using current substitution cipher guess
    s = decrypt(ciphertext, plaintext_alphabet, cipher_alphabet, blacklist={})
    message_length = len(s)
    
    # Count tri-, bi- and uni-gram instances in decrypted guess
    for c in range(message_length-2):
        trigram_counts[(s[c],s[c+1],s[c+2])] +=1
        bigram_counts[(s[c],s[c+1])] +=1
        unigram_counts[s[c]] += 1

    # Add last bigram in the line
    unigram_counts[(s[message_length-2],s[message_length-1])] += 1     
                
    # Add last two unigrams in the line
    unigram_counts[s[message_length-2]] += 1 
    unigram_counts[s[message_length-1]] += 1 

    # Construct composite likelihood using trigrams and reference_freq
    clogl_tri = 0
    for trigram, count in trigram_counts.items():
        clogl_tri += count*np.log(reference_freq['pop_trigram_freqs'].setdefault(trigram, epsilon))
        
    # Construct composite likelihood using bigrams and reference_freq
    clogl_bi = 0
    for bigram, count in bigram_counts.items():
        clogl_bi += count*np.log(reference_freq['pop_bigram_freqs'].setdefault(bigram, epsilon))
        
    # Construct composite likelihood using unigrams and reference_freq
    clogl_uni = 0
    for unigram, count in unigram_counts.items():
        clogl_uni += count*np.log(reference_freq['pop_unigram_freqs'].setdefault(unigram, epsilon))
      
    return {'clogl_tri' : clogl_tri, 'clogl_bi' : clogl_bi, 'clogl_uni' : clogl_uni}

In [5]:
# Directory where Dune text file is located
data =  '/Users/bgraham/Dropbox/Teaching/Berkeley_Courses/Ec143/Ec143_Spring2026/Data/'

In [6]:
dune_freq = process_text(data+'Dune.txt')

1000 lines read.
2000 lines read.
3000 lines read.
4000 lines read.
5000 lines read.
6000 lines read.
7000 lines read.
8000 lines read.


In [7]:
# Define and clean message
message = "I must not fear. Fear is the mind-killer. Fear is the little-death that brings total obliteration. I will face my fear. I will permit it to pass over me and through me. And when it has gone past I will turn the inner eye to see its path. Where the fear has gone there will be nothing. Only I will remain."
ignore_list = '[^A-Z .]'
pattern = re.compile(ignore_list)
message_cleaned = pattern.sub('',message.upper())
message_cleaned

# Encrypt message using a randomly chosen simpler substitution cipher
english_alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ ."       # define plaintext alphabet
tmp = list(english_alphabet)                            # turn it into a list and randomly shuffle
random.shuffle(tmp)
cipher_alphabet = "".join(tmp)                          # define ciphertext alphabet
message_encrypted = encrypt(message_cleaned, english_alphabet, cipher_alphabet)

print("")
print("Original (cleaned) message:  ")
print("-----------------------------")
print(message_cleaned)
print("")
print("Encrypted message:           ")
print("-----------------------------")
print(message_encrypted)


# true decryption key
dec_key = dict(zip(list(cipher_alphabet),list(english_alphabet)))
print("")
print("Simple substitution cipher:  ")
print("-----------------------------")
print(dec_key)




Original (cleaned) message:  
-----------------------------
I MUST NOT FEAR. FEAR IS THE MINDKILLER. FEAR IS THE LITTLEDEATH THAT BRINGS TOTAL OBLITERATION. I WILL FACE MY FEAR. I WILL PERMIT IT TO PASS OVER ME AND THROUGH ME. AND WHEN IT HAS GONE PAST I WILL TURN THE INNER EYE TO SEE ITS PATH. WHERE THE FEAR HAS GONE THERE WILL BE NOTHING. ONLY I WILL REMAIN.

Encrypted message:           
-----------------------------
AXZKJCXW.CXRVGLNXRVGLXAJXCPVXZAWYHAFFVLNXRVGLXAJXCPVXFACCFVYVGCPXCPGCX LAWEJXC.CGFX. FACVLGCA.WNXAXTAFFXRGMVXZSXRVGLNXAXTAFFXOVLZACXACXC.XOGJJX.BVLXZVXGWYXCPL.KEPXZVNXGWYXTPVWXACXPGJXE.WVXOGJCXAXTAFFXCKLWXCPVXAWWVLXVSVXC.XJVVXACJXOGCPNXTPVLVXCPVXRVGLXPGJXE.WVXCPVLVXTAFFX VXW.CPAWENX.WFSXAXTAFFXLVZGAWN

Simple substitution cipher:  
-----------------------------
{'G': 'A', ' ': 'B', 'M': 'C', 'Y': 'D', 'V': 'E', 'R': 'F', 'E': 'G', 'P': 'H', 'A': 'I', 'U': 'J', 'H': 'K', 'F': 'L', 'Z': 'M', 'W': 'N', '.': 'O', 'O': 'P', 'I': 'Q', 'L': 'R', 'J': 'S', 'C': 'T', 'K': 'U', 

## Cracking an encrypted message

In [8]:
print("")
print("Encrypted message:           ")
print("-----------------------------")
print(message_encrypted)



Encrypted message:           
-----------------------------
AXZKJCXW.CXRVGLNXRVGLXAJXCPVXZAWYHAFFVLNXRVGLXAJXCPVXFACCFVYVGCPXCPGCX LAWEJXC.CGFX. FACVLGCA.WNXAXTAFFXRGMVXZSXRVGLNXAXTAFFXOVLZACXACXC.XOGJJX.BVLXZVXGWYXCPL.KEPXZVNXGWYXTPVWXACXPGJXE.WVXOGJCXAXTAFFXCKLWXCPVXAWWVLXVSVXC.XJVVXACJXOGCPNXTPVLVXCPVXRVGLXPGJXE.WVXCPVLVXTAFFX VXW.CPAWENX.WFSXAXTAFFXLVZGAWN


In [10]:
plaintext_alphabet = english_alphabet
reference_freq = dune_freq
ciphertext = message_encrypted
iters = 100000

# Counter for the number of accepted swaps
accepted_swaps = 0

# Dictionary to store MCMC output
mcmc_output = {'s' : [], 'log_likelihood' : [], 'cipher' : []}

# initial cipher key guess (random) and likelihood value
tmp = list(plaintext_alphabet)   
random.shuffle(tmp)
current_plaintext_alphabet_guess = "".join(tmp)  
logL_s = composite_ciper_logl(ciphertext, current_plaintext_alphabet_guess, cipher_alphabet, reference_freq)

# current current cipher key
current_decryption_key = dict(zip(list(cipher_alphabet),list(current_plaintext_alphabet_guess)))

# add initial values to dictionary
mcmc_output['s'].append(0)
mcmc_output['log_likelihood'].append(logL_s['clogl_bi'])
mcmc_output['cipher'].append(current_decryption_key)

for s in range(0,iters):
   
    # generate proposal cipher by swapping two characters in current cipher
    a, b = np.random.choice(list(cipher_alphabet), 2, replace=True)
    proposed_decryption_key    = current_decryption_key.copy()
    proposed_decryption_key[a] = current_decryption_key[b]
    proposed_decryption_key[b] = current_decryption_key[a]
    proposed_plaintext_alphabet_guess = "".join(list(proposed_decryption_key.values()))


    # compute difference in current and proposed composite logL
    logL_proposed = composite_ciper_logl(ciphertext, proposed_plaintext_alphabet_guess, cipher_alphabet, reference_freq)
    log_alpha = logL_proposed['clogl_bi'] - logL_s['clogl_bi']

    # Decide to accept new proposal
    V = np.random.exponential(scale=1.0)
    if V >= -log_alpha:
        logL_s = logL_proposed
        current_plaintext_alphabet_guess = proposed_plaintext_alphabet_guess
        current_decryption_key = dict(zip(list(cipher_alphabet),list(proposed_plaintext_alphabet_guess)))
        accepted_swaps += 1
    
    # add current iteration of output to dictionary
    mcmc_output['s'].append(s)
    mcmc_output['log_likelihood'].append(logL_s['clogl_bi'])
    mcmc_output['cipher'].append(current_decryption_key)

    # Print out progress
    if s % 500 == 0: 
        print("")
        print("iter:", s, decrypt(ciphertext, current_plaintext_alphabet_guess, cipher_alphabet, blacklist={}))
    


iter: 0 EVDL HVASHVRJQW.VRJQWVE VHPJVDEATCEFFJW.VRJQWVE VHPJVFEHHFJTJQHPVHPQHVIWEAZ VHSHQFVSIFEHJWQHESA.VEVGEFFVRQKJVDMVRJQW.VEVGEFFVYJWDEHVEHVHSVYQ  VSXJWVDJVQATVHPWSLZPVDJ.VQATVGPJAVEHVPQ VZSAJVYQ HVEVGEFFVHLWAVHPJVEAAJWVJMJVHSV JJVEH VYQHP.VGPJWJVHPJVRJQWVPQ VZSAJVHPJWJVGEFFVIJVASHPEAZ.VSAFMVEVGEFFVWJDQEA.

iter: 500 A WKRS MUS BEONT BEON AR SIE WAMP.ALLENT BEON AR SIE LASSLEPEOSI SIOS GNAMYR SUSOL UGLASENOSAUMT A CALL BOVE WH BEONT A CALL FENWAS AS SU FORR UDEN WE OMP SINUKYI WET OMP CIEM AS IOR YUME FORS A CALL SKNM SIE AMMEN EHE SU REE ASR FOSIT CIENE SIE BEON IOR YUME SIENE CALL GE MUSIAMYT UMLH A CALL NEWOAMT

iter: 1000 A CYDS RIS WEONG WEON AD STE CARMPALLENG WEON AD STE LASSLEMEOST STOS KNARUD SISOL IKLASENOSAIRG A HALL WOVE C. WEONG A HALL FENCAS AS SI FODD IBEN CE ORM STNIYUT CEG ORM HTER AS TOD UIRE FODS A HALL SYNR STE ARREN E.E SI DEE ASD FOSTG HTENE STE WEON TOD UIRE STENE HALL KE RISTARUG IRL. A HALL NECOARG

iter: 1500 A CUDS ROS HEING HEIN AD STE CARMBALLENG HEIN A


iter: 14500 I MURT NOT PEAD. PEAD IR THE MINGKISSED. PEAD IR THE SITTSEGEATH THAT LDINCR TOTAS OLSITEDATION. I WISS PAVE MY PEAD. I WISS BEDMIT IT TO BARR OFED ME ANG THDOUCH ME. ANG WHEN IT HAR CONE BART I WISS TUDN THE INNED EYE TO REE ITR BATH. WHEDE THE PEAD HAR CONE THEDE WISS LE NOTHINC. ONSY I WISS DEMAIN.

iter: 15000 I MURT NOT PEAD. PEAD IR THE MINCKISSED. PEAD IR THE SITTSECEATH THAT LDINGR TOTAS OLSITEDATION. I WISS PAVE MY PEAD. I WISS BEDMIT IT TO BARR OFED ME ANC THDOUGH ME. ANC WHEN IT HAR GONE BART I WISS TUDN THE INNED EYE TO REE ITR BATH. WHEDE THE PEAD HAR GONE THEDE WISS LE NOTHING. ONSY I WISS DEMAIN.

iter: 15500 I MURT NOT BEAD. BEAD IR THE MINCKISSED. BEAD IR THE SITTSECEATH THAT LDINGR TOTAS OLSITEDATION. I WISS BAVE MY BEAD. I WISS PEDMIT IT TO PARR OFED ME ANC THDOUGH ME. ANC WHEN IT HAR GONE PART I WISS TUDN THE INNED EYE TO REE ITR PATH. WHEDE THE BEAD HAR GONE THEDE WISS LE NOTHING. ONSY I WISS DEMAIN.

iter: 16000 I MURT NOT BEAD. BEAD IR THE MINGFISSED


iter: 29500 I MUST NOT BEAR. BEAR IS THE MINDVILLER. BEAR IS THE LITTLEDEATH THAT FRINGS TOTAL OFLITERATION. I WILL BAKE MY BEAR. I WILL PERMIT IT TO PASS OCER ME AND THROUGH ME. AND WHEN IT HAS GONE PAST I WILL TURN THE INNER EYE TO SEE ITS PATH. WHERE THE BEAR HAS GONE THERE WILL FE NOTHING. ONLY I WILL REMAIN.

iter: 30000 I MUST NOT BEAR. BEAR IS THE MINDVILLER. BEAR IS THE LITTLEDEATH THAT PRINGS TOTAL OPLITERATION. I WILL BAKE MY BEAR. I WILL CERMIT IT TO CASS OFER ME AND THROUGH ME. AND WHEN IT HAS GONE CAST I WILL TURN THE INNER EYE TO SEE ITS CATH. WHERE THE BEAR HAS GONE THERE WILL PE NOTHING. ONLY I WILL REMAIN.

iter: 30500 I MUST NOT BEAR. BEAR IS THE MINDVILLER. BEAR IS THE LITTLEDEATH THAT PRINGS TOTAL OPLITERATION. I WILL BAKE MY BEAR. I WILL CERMIT IT TO CASS OFER ME AND THROUGH ME. AND WHEN IT HAS GONE CAST I WILL TURN THE INNER EYE TO SEE ITS CATH. WHERE THE BEAR HAS GONE THERE WILL PE NOTHING. ONLY I WILL REMAIN.

iter: 31000 I MUST NOT BEAR. BEAR IS THE MINDFILLER


iter: 45500 I MUST NOT BEAR. BEAR IS THE MINDVILLER. BEAR IS THE LITTLEDEATH THAT PRINGS TOTAL OPLITERATION. I WILL BAKE MY BEAR. I WILL CERMIT IT TO CASS OFER ME AND THROUGH ME. AND WHEN IT HAS GONE CAST I WILL TURN THE INNER EYE TO SEE ITS CATH. WHERE THE BEAR HAS GONE THERE WILL PE NOTHING. ONLY I WILL REMAIN.

iter: 46000 I MUST NOT BEAR. BEAR IS THE MINDVILLER. BEAR IS THE LITTLEDEATH THAT FRINGS TOTAL OFLITERATION. I WILL BAKE MY BEAR. I WILL CERMIT IT TO CASS OPER ME AND THROUGH ME. AND WHEN IT HAS GONE CAST I WILL TURN THE INNER EYE TO SEE ITS CATH. WHERE THE BEAR HAS GONE THERE WILL FE NOTHING. ONLY I WILL REMAIN.

iter: 46500 I MUST NOT BEAR. BEAR IS THE MINDCILLER. BEAR IS THE LITTLEDEATH THAT FRINGS TOTAL OFLITERATION. I WILL BAKE MY BEAR. I WILL PERMIT IT TO PASS OVER ME AND THROUGH ME. AND WHEN IT HAS GONE PAST I WILL TURN THE INNER EYE TO SEE ITS PATH. WHERE THE BEAR HAS GONE THERE WILL FE NOTHING. ONLY I WILL REMAIN.

iter: 47000 I MUST NOT BEAR. BEAR IS THE MINDVILLER


iter: 67500 I MUST NOT BEAR. BEAR IS THE MINDVILLER. BEAR IS THE LITTLEDEATH THAT FRINGS TOTAL OFLITERATION. I WILL BAPE MY BEAR. I WILL CERMIT IT TO CASS OKER ME AND THROUGH ME. AND WHEN IT HAS GONE CAST I WILL TURN THE INNER EYE TO SEE ITS CATH. WHERE THE BEAR HAS GONE THERE WILL FE NOTHING. ONLY I WILL REMAIN.

iter: 68000 I MUST NOT BEAR. BEAR IS THE MINDVILLER. BEAR IS THE LITTLEDEATH THAT FRINGS TOTAL OFLITERATION. I WILL BAKE MY BEAR. I WILL CERMIT IT TO CASS OPER ME AND THROUGH ME. AND WHEN IT HAS GONE CAST I WILL TURN THE INNER EYE TO SEE ITS CATH. WHERE THE BEAR HAS GONE THERE WILL FE NOTHING. ONLY I WILL REMAIN.

iter: 68500 I MUST NOT BEAR. BEAR IS THE MINDKILLER. BEAR IS THE LITTLEDEATH THAT FRINGS TOTAL OFLITERATION. I WILL BAVE MY BEAR. I WILL CERMIT IT TO CASS OPER ME AND THROUGH ME. AND WHEN IT HAS GONE CAST I WILL TURN THE INNER EYE TO SEE ITS CATH. WHERE THE BEAR HAS GONE THERE WILL FE NOTHING. ONLY I WILL REMAIN.

iter: 69000 I MUST NOT BEAR. BEAR IS THE MINDVILLER


iter: 83500 I MUST NOT BEAR. BEAR IS THE MINDVILLER. BEAR IS THE LITTLEDEATH THAT PRINGS TOTAL OPLITERATION. I WILL BAKE MY BEAR. I WILL CERMIT IT TO CASS OFER ME AND THROUGH ME. AND WHEN IT HAS GONE CAST I WILL TURN THE INNER EYE TO SEE ITS CATH. WHERE THE BEAR HAS GONE THERE WILL PE NOTHING. ONLY I WILL REMAIN.

iter: 84000 I MUST NOT BEAR. BEAR IS THE MINDVILLER. BEAR IS THE LITTLEDEATH THAT FRINGS TOTAL OFLITERATION. I WILL BAKE MY BEAR. I WILL CERMIT IT TO CASS OPER ME AND THROUGH ME. AND WHEN IT HAS GONE CAST I WILL TURN THE INNER EYE TO SEE ITS CATH. WHERE THE BEAR HAS GONE THERE WILL FE NOTHING. ONLY I WILL REMAIN.

iter: 84500 I MUST NOT BEAR. BEAR IS THE MINDVILLER. BEAR IS THE LITTLEDEATH THAT FRINGS TOTAL OFLITERATION. I WILL BAPE MY BEAR. I WILL CERMIT IT TO CASS OKER ME AND THROUGH ME. AND WHEN IT HAS GONE CAST I WILL TURN THE INNER EYE TO SEE ITS CATH. WHERE THE BEAR HAS GONE THERE WILL FE NOTHING. ONLY I WILL REMAIN.

iter: 85000 I MUST NOT BEAR. BEAR IS THE MINDKILLER

In [11]:
mcmc_output = pd.DataFrame(mcmc_output)
mcmc_output[0:5]

,s,log_likelihood,cipher
0,0,-5800.826471,"{'G': 'Q', ' ': 'I', 'M': 'K', 'Y': 'T', 'V': ..."
1,0,-5800.826471,"{'G': 'Q', ' ': 'I', 'M': 'K', 'Y': 'T', 'V': ..."
2,1,-5602.080698,"{'G': 'Q', ' ': 'I', 'M': 'K', 'Y': 'T', 'V': ..."
3,2,-5577.859915,"{'G': 'Q', ' ': 'I', 'M': 'K', 'Y': 'T', 'V': ..."
4,3,-5488.644152,"{'G': 'Q', ' ': 'I', 'M': 'K', 'Y': 'T', 'V': ..."


In [12]:
top_cipher_guess = mcmc_output['log_likelihood'].idxmax()

print("Cipher guess with highest posterior probability: ")
print(mcmc_output['cipher'].loc[top_cipher_guess])
plaintext_alphabet_top_guess = "".join(list(mcmc_output['cipher'].loc[top_cipher_guess].values()))
decrypt(message_encrypted, plaintext_alphabet_top_guess, cipher_alphabet, blacklist={})    

Cipher guess with highest posterior probability: 
{'G': 'A', ' ': 'P', 'M': 'K', 'Y': 'D', 'V': 'E', 'R': 'B', 'E': 'G', 'P': 'H', 'A': 'I', 'U': 'Z', 'H': 'V', 'F': 'L', 'Z': 'M', 'W': 'N', '.': 'O', 'O': 'C', 'I': 'X', 'L': 'R', 'J': 'S', 'C': 'T', 'K': 'U', 'B': 'F', 'T': 'W', 'D': 'J', 'S': 'Y', 'Q': 'Q', 'X': ' ', 'N': '.'}


'I MUST NOT BEAR. BEAR IS THE MINDVILLER. BEAR IS THE LITTLEDEATH THAT PRINGS TOTAL OPLITERATION. I WILL BAKE MY BEAR. I WILL CERMIT IT TO CASS OFER ME AND THROUGH ME. AND WHEN IT HAS GONE CAST I WILL TURN THE INNER EYE TO SEE ITS CATH. WHERE THE BEAR HAS GONE THERE WILL PE NOTHING. ONLY I WILL REMAIN.'